In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import datetime

#Librerias para visualizacion
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:

#labels_training = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/labels_training_set.csv',
#                              low_memory= False)
#events_data = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/events_up_to_01062018.csv', 
#                              low_memory= False)

labels_training = pd.read_csv('D:/DATOS/RELOADED/csv/labels_training_set.csv',
                               low_memory= False)
events_data = pd.read_csv('D:/DATOS/RELOADED/csv/events_up_to_01062018.csv', 
                               low_memory= False)

In [3]:
#Data Frame de features
features = pd.DataFrame()

In [4]:
events_data.head()

timestamp           event    person  url      sku  \
0  2018-05-18 00:11:59  viewed product  4886f805  NaN   9288.0   
1  2018-05-18 00:11:27  viewed product  ad93850f  NaN    304.0   
2  2018-05-18 00:11:16  viewed product  0297fc1e  NaN   6888.0   
3  2018-05-18 00:11:14  viewed product  2d681dd8  NaN  11890.0   
4  2018-05-18 00:11:09  viewed product  cccea85e  NaN   7517.0   

                     model  condition storage           color skus  \
0  Samsung Galaxy J7 Prime  Excelente    32GB         Dourado  NaN   
1                iPhone 5s  Muito Bom    32GB  Cinza espacial  NaN   
2                iPhone 6S  Muito Bom    64GB        Prateado  NaN   
3                 iPhone 7        Bom   128GB        Vermelho  NaN   
4              LG G4 H818P  Excelente    32GB          Branco  NaN   

        ...       search_engine channel new_vs_returning city region country  \
0       ...                 NaN     NaN              NaN  NaN    NaN     NaN   
1       ...                 NaN     NaN              NaN  NaN    NaN     NaN   
2       ...                 NaN     NaN              NaN  NaN    NaN     NaN   
3       ...                 NaN     NaN              NaN  NaN    NaN     NaN   
4       ...                 NaN     NaN              NaN  NaN    NaN     NaN   

  device_type screen_resolution operating_system_version browser_version  
0         NaN               NaN                      NaN             NaN  
1         NaN               NaN                      NaN             NaN  
2         NaN               NaN                      NaN             NaN  
3         NaN               NaN                      NaN             NaN  
4         NaN               NaN                      NaN             NaN  

[5 rows x 23 columns]

In [5]:
events_data.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'],
      dtype='object')

In [6]:
labels_training.head()

person  label
0  0566e9c1      0
1  6ec7ee77      0
2  abe7a2fb      0
3  34728364      0
4  87ed62de      0

In [7]:
#En features tengo los usuarios sin repetir
features['person'] = events_data['person'].unique()

In [8]:
events_data['hour'] = pd.to_datetime(events_data['timestamp']).dt.hour
events_data['date'] = pd.to_datetime(events_data['timestamp']).dt.date

In [9]:
map_hora = {0:24}

In [10]:
events_data.replace({'hour':map_hora}, inplace = True)

In [11]:
events_data.head()

timestamp           event    person  url      sku  \
0  2018-05-18 00:11:59  viewed product  4886f805  NaN   9288.0   
1  2018-05-18 00:11:27  viewed product  ad93850f  NaN    304.0   
2  2018-05-18 00:11:16  viewed product  0297fc1e  NaN   6888.0   
3  2018-05-18 00:11:14  viewed product  2d681dd8  NaN  11890.0   
4  2018-05-18 00:11:09  viewed product  cccea85e  NaN   7517.0   

                     model  condition storage           color skus  \
0  Samsung Galaxy J7 Prime  Excelente    32GB         Dourado  NaN   
1                iPhone 5s  Muito Bom    32GB  Cinza espacial  NaN   
2                iPhone 6S  Muito Bom    64GB        Prateado  NaN   
3                 iPhone 7        Bom   128GB        Vermelho  NaN   
4              LG G4 H818P  Excelente    32GB          Branco  NaN   

      ...     new_vs_returning city region country device_type  \
0     ...                  NaN  NaN    NaN     NaN         NaN   
1     ...                  NaN  NaN    NaN     NaN         NaN   
2     ...                  NaN  NaN    NaN     NaN         NaN   
3     ...                  NaN  NaN    NaN     NaN         NaN   
4     ...                  NaN  NaN    NaN     NaN         NaN   

  screen_resolution operating_system_version browser_version hour        date  
0               NaN                      NaN             NaN   24  2018-05-18  
1               NaN                      NaN             NaN   24  2018-05-18  
2               NaN                      NaN             NaN   24  2018-05-18  
3               NaN                      NaN             NaN   24  2018-05-18  
4               NaN                      NaN             NaN   24  2018-05-18  

[5 rows x 25 columns]

In [12]:
events_data['weekday'] = pd.to_datetime(events_data['date']).dt.day_name()

In [13]:
events_data.head()

timestamp           event    person  url      sku  \
0  2018-05-18 00:11:59  viewed product  4886f805  NaN   9288.0   
1  2018-05-18 00:11:27  viewed product  ad93850f  NaN    304.0   
2  2018-05-18 00:11:16  viewed product  0297fc1e  NaN   6888.0   
3  2018-05-18 00:11:14  viewed product  2d681dd8  NaN  11890.0   
4  2018-05-18 00:11:09  viewed product  cccea85e  NaN   7517.0   

                     model  condition storage           color skus   ...    \
0  Samsung Galaxy J7 Prime  Excelente    32GB         Dourado  NaN   ...     
1                iPhone 5s  Muito Bom    32GB  Cinza espacial  NaN   ...     
2                iPhone 6S  Muito Bom    64GB        Prateado  NaN   ...     
3                 iPhone 7        Bom   128GB        Vermelho  NaN   ...     
4              LG G4 H818P  Excelente    32GB          Branco  NaN   ...     

  city region country device_type screen_resolution operating_system_version  \
0  NaN    NaN     NaN         NaN               NaN                      NaN   
1  NaN    NaN     NaN         NaN               NaN                      NaN   
2  NaN    NaN     NaN         NaN               NaN                      NaN   
3  NaN    NaN     NaN         NaN               NaN                      NaN   
4  NaN    NaN     NaN         NaN               NaN                      NaN   

  browser_version hour        date weekday  
0             NaN   24  2018-05-18  Friday  
1             NaN   24  2018-05-18  Friday  
2             NaN   24  2018-05-18  Friday  
3             NaN   24  2018-05-18  Friday  
4             NaN   24  2018-05-18  Friday  

[5 rows x 26 columns]

**Cantidad de conversiones por usuario**

In [14]:
conversions_user = pd.DataFrame()

In [15]:
conversions_user = events_data.loc[events_data['event'] == 'conversion',['person']]

In [16]:

conversions_user= pd.DataFrame(conversions_user['person'].value_counts())

In [17]:
conversions_user = conversions_user.reset_index()
conversions_user.rename({'index':'person', 'person':'cantidad de conversiones'}, axis = 1, inplace = True)
conversions_user.head()

person  cantidad de conversiones
0  5059f7fd                       129
1  888a11a2                        44
2  0280a93d                        32
3  b4cdf2b5                        30
4  32c21f9e                        21

In [18]:
features = features.merge(conversions_user, on = 'person', how = 'left')

In [19]:
features.head()

person  cantidad de conversiones
0  4886f805                       NaN
1  ad93850f                       NaN
2  0297fc1e                       NaN
3  2d681dd8                       NaN
4  cccea85e                       NaN

In [20]:
#features['cantidad de conversiones'] = features['cantidad de conversiones'].fillna(0) 

In [21]:
features.head(20)

person  cantidad de conversiones
0   4886f805                       NaN
1   ad93850f                       NaN
2   0297fc1e                       NaN
3   2d681dd8                       NaN
4   cccea85e                       NaN
5   4c8a8b93                       NaN
6   1b9f7cf6                       NaN
7   29ebb414                       NaN
8   de8fe91b                       NaN
9   45baf068                       NaN
10  99abca5a                       2.0
11  3dc1950f                       NaN
12  8ea4c165                       NaN
13  64f45e8d                       NaN
14  d8cfe234                       1.0
15  ade6b3bf                       NaN
16  d73ae9ef                       NaN
17  d6bc64df                       NaN
18  ec9c4059                       1.0
19  b2cde41a                       NaN

In [22]:
events_data['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

**Cantidad de veces que cada usuario vio un producto**

In [23]:
view_user = pd.DataFrame()
view_user = events_data.loc[events_data['event'] == 'viewed product',['person']]
view_user= pd.DataFrame(view_user['person'].value_counts())
view_user = view_user.reset_index()
view_user.rename({'index':'person', 'person':'cantidad de veces que vio un producto'}, axis = 1, inplace = True)
view_user.head()

person  cantidad de veces que vio un producto
0  6abd2bf1                                   2355
1  b1f4dbf6                                   2233
2  8fb4929e                                   1912
3  06ed04d6                                   1881
4  bfb74b38                                   1782

In [24]:
features.head(10)

person  cantidad de conversiones
0  4886f805                       NaN
1  ad93850f                       NaN
2  0297fc1e                       NaN
3  2d681dd8                       NaN
4  cccea85e                       NaN
5  4c8a8b93                       NaN
6  1b9f7cf6                       NaN
7  29ebb414                       NaN
8  de8fe91b                       NaN
9  45baf068                       NaN

In [25]:
features = features.merge(view_user, on = 'person', how = 'left')
#features['cantidad de veces que vio un producto'] = features['cantidad de veces que vio un producto'].fillna(0) 

In [26]:
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto
0  4886f805                       NaN                                    4.0
1  ad93850f                       NaN                                   20.0
2  0297fc1e                       NaN                                  404.0
3  2d681dd8                       NaN                                   13.0
4  cccea85e                       NaN                                  739.0

**Cantidad de visitas a la pagina de cada usuario**

In [27]:
visit_user = pd.DataFrame()
visit_user = events_data.loc[events_data['event'] == 'visited site',['person']]
visit_user= pd.DataFrame(visit_user['person'].value_counts())
visit_user = visit_user.reset_index()
visit_user.rename({'index':'person', 'person':'cantidad de visitas a la pagina'}, axis = 1, inplace = True)
visit_user.head()

person  cantidad de visitas a la pagina
0  5059f7fd                              295
1  02f14240                              294
2  67bdc946                              210
3  7ac0c607                              208
4  9b3b43aa                              184

In [28]:
features = features.merge(visit_user, on = 'person', how = 'left')
#features['cantidad de visitas a la pagina'] = features['cantidad de visitas a la pagina'].fillna(0)
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  
0                              1.0  
1                              5.0  
2                             95.0  
3                              2.0  
4                             22.0

In [29]:
features['cantidad de veces que vio un producto'].isnull().any()

True

In [30]:
features['cantidad de visitas a la pagina'].isnull().any()

True

**Cantidad de visitas a la pagina de cada usuario**

In [31]:
check_user = pd.DataFrame()
check_user = events_data.loc[events_data['event'] == 'checkout',['person']]
check_user= pd.DataFrame(check_user['person'].value_counts())
check_user = check_user.reset_index()
check_user.rename({'index':'person', 'person':'cantidad de checkouts'}, axis = 1, inplace = True)
check_user.head()

person  cantidad de checkouts
0  5059f7fd                    197
1  a1b25eb0                    101
2  043ec736                     66
3  453b63ca                     65
4  e1443dd4                     64

In [32]:
features = features.merge(check_user, on = 'person', how = 'left')
#features['cantidad de checkouts'] = features['cantidad de checkouts'].fillna(0)
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  
0                              1.0                    1.0  
1                              5.0                    1.0  
2                             95.0                    7.0  
3                              2.0                    1.0  
4                             22.0                    1.0

**Conversiones por hora del dia**

In [33]:
conversions_hour = pd.DataFrame()
conversions_hour = events_data.loc[events_data['event'] == 'conversion',['hour']]

In [34]:
conversions_hour = pd.DataFrame(conversions_hour['hour'].value_counts()/conversions_hour.shape[0])
conversions_hour.reset_index()


index      hour
0      21  0.069525
1      18  0.066986
2      17  0.065717
3      19  0.063884
4      15  0.062615
5      24  0.061486
6      16  0.061204
7      20  0.059653
8      23  0.059653
9      13  0.059371
10     22  0.058666
11     14  0.056410
12      1  0.055563
13     12  0.043294
14      2  0.041743
15     11  0.032012
16      3  0.028346
17     10  0.015231
18      4  0.013397
19      5  0.007192
20      9  0.006910
21      6  0.004372
22      7  0.003949
23      8  0.002820

Aproximadamente el 66% de las conversiones se realizaron enntre las 13 y las 24 horas. 
Veo la cantidad de visitas a la pagina de cada usuario para ese horario

In [35]:
horario = pd.DataFrame()
horario = events_data.loc[(events_data['hour'] > 12)& 
                          (events_data['event'] == 'visited site'),['person']]
horario.head()

person
2136629  4640420b
2136632  4640420b
2136633  4640420b
2136634  4640420b
2136635  4640420b

In [36]:
horario= pd.DataFrame(horario['person'].value_counts())
horario = horario.reset_index()
horario.rename({'index':'person', 'person':'visitaron el sitio entre las 13hs y 24hs'}, axis = 1, inplace = True)
horario.head()

person  visitaron el sitio entre las 13hs y 24hs
0  5059f7fd                                       229
1  02f14240                                       199
2  7ac0c607                                       163
3  ffee0f18                                       148
4  67bdc946                                       148

In [37]:
features = features.merge(horario, on = 'person', how = 'left')


In [38]:
features.head(20)

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0   4886f805                       NaN                                    4.0   
1   ad93850f                       NaN                                   20.0   
2   0297fc1e                       NaN                                  404.0   
3   2d681dd8                       NaN                                   13.0   
4   cccea85e                       NaN                                  739.0   
5   4c8a8b93                       NaN                                  177.0   
6   1b9f7cf6                       NaN                                    9.0   
7   29ebb414                       NaN                                    8.0   
8   de8fe91b                       NaN                                   27.0   
9   45baf068                       NaN                                    6.0   
10  99abca5a                       2.0                                  630.0   
11  3dc1950f                       NaN                                  595.0   
12  8ea4c165                       NaN                                   97.0   
13  64f45e8d                       NaN                                   32.0   
14  d8cfe234                       1.0                                   39.0   
15  ade6b3bf                       NaN                                   92.0   
16  d73ae9ef                       NaN                                    5.0   
17  d6bc64df                       NaN                                  155.0   
18  ec9c4059                       1.0                                   25.0   
19  b2cde41a                       NaN                                   27.0   

    cantidad de visitas a la pagina  cantidad de checkouts  \
0                               1.0                    1.0   
1                               5.0                    1.0   
2                              95.0                    7.0   
3                               2.0                    1.0   
4                              22.0                    1.0   
5                              20.0                    2.0   
6                               3.0                    1.0   
7                               4.0                    1.0   
8                               2.0                    1.0   
9                               1.0                    1.0   
10                             56.0                   18.0   
11                             30.0                    3.0   
12                              7.0                    3.0   
13                              3.0                    2.0   
14                              4.0                    3.0   
15                             13.0                    1.0   
16                              1.0                    1.0   
17                              7.0                    2.0   
18                              7.0                    2.0   
19                              4.0                    1.0   

    visitaron el sitio entre las 13hs y 24hs  
0                                        1.0  
1                                        4.0  
2                                       64.0  
3                                        2.0  
4                                       18.0  
5                                       13.0  
6                                        2.0  
7                                        4.0  
8                                        2.0  
9                                        1.0  
10                                      45.0  
11                                      24.0  
12                                       6.0  
13                                       3.0  
14                                       4.0  
15                                       7.0  
16                                       1.0  
17                                       2.0  
18                                       3.0  
19                                       4.0

**Veo cuantas veces vio cada uno de los 5 modelos mas vendidos segun lo obtenido en la exploracion del tp1**

Del menos la mas vendido
iPhone 6S, Samsung Galaxy S6 Flat, iPhone 6 , iPhone 5S, Samsung Galaxy J5

Veo por usuario cuantas veces vio uno de estos productos

In [39]:
viewed_product_event = pd.DataFrame()
viewed_product_event = events_data.loc[events_data['event'] == 'viewed product',['person', 'model']]

In [40]:
#samsung galaxy j5
samsung_j5 = pd.DataFrame()
samsung_j5 = viewed_product_event.loc[viewed_product_event['model'] == 'Samsung Galaxy J5',['person']]

In [41]:
samsung_j5= pd.DataFrame(samsung_j5['person'].value_counts())
samsung_j5 = samsung_j5.reset_index()
samsung_j5.rename({'index':'person', 'person':'cantidad de vistas del J5'}, axis = 1, inplace = True)
samsung_j5.head()

person  cantidad de vistas del J5
0  5059f7fd                        149
1  1775ba85                        140
2  86742710                        131
3  e68cc89a                        110
4  9c743b65                        107

In [42]:
features = features.merge(samsung_j5, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  
0                                       1.0                        NaN  
1                                       4.0                        NaN  
2                                      64.0                        NaN  
3                                       2.0                        NaN  
4                                      18.0                        NaN

In [43]:
#iphone 5s
iPhone_5S = pd.DataFrame()
iPhone_5S = viewed_product_event.loc[viewed_product_event['model'] == 'iPhone 5s',['person']]
iPhone_5S= pd.DataFrame(iPhone_5S['person'].value_counts())
iPhone_5S = iPhone_5S.reset_index()
iPhone_5S.rename({'index':'person', 'person':'cantidad de vistas del iphone 5s'}, axis = 1, inplace = True)
iPhone_5S.head()

person  cantidad de vistas del iphone 5s
0  b1f4dbf6                               832
1  0d5f66ed                               495
2  360feb42                               406
3  c1aea85e                               394
4  d6db0770                               376

In [44]:
features = features.merge(iPhone_5S, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  
0                               NaN  
1                              17.0  
2                               1.0  
3                               NaN  
4                               2.0

In [45]:
#iPhone 6
samsung_s6_flat = pd.DataFrame()
samsung_s6_flat = viewed_product_event.loc[viewed_product_event['model'] == 'Samsung Galaxy S6 Flat',['person']]
samsung_s6_flat= pd.DataFrame(samsung_s6_flat['person'].value_counts())
samsung_s6_flat = samsung_s6_flat.reset_index()
samsung_s6_flat.rename({'index':'person', 'person':'cantidad de vistas del samsung s6 flat'}, axis = 1, inplace = True)
samsung_s6_flat.head()

person  cantidad de vistas del samsung s6 flat
0  a879f712                                     238
1  439d167c                                     225
2  5961db59                                     146
3  000e4d9e                                     139
4  595b9b50                                     117

In [46]:
features = features.merge(samsung_s6_flat, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  
0                               NaN                                     NaN  
1                              17.0                                     NaN  
2                               1.0                                     NaN  
3                               NaN                                     NaN  
4                               2.0                                     1.0

In [47]:
#iPhone 6
iPhone_6 = pd.DataFrame()
iPhone_6 = viewed_product_event.loc[viewed_product_event['model'] == 'iPhone 6',['person']]
iPhone_6= pd.DataFrame(iPhone_6['person'].value_counts())
iPhone_6 = iPhone_6.reset_index()
iPhone_6.rename({'index':'person', 'person':'cantidad de vistas del iphone 6'}, axis = 1, inplace = True)
iPhone_6.head()

person  cantidad de vistas del iphone 6
0  6abd2bf1                              603
1  9bf968c5                              558
2  c86f17ad                              417
3  09ebd175                              341
4  edbdffb9                              318

In [48]:
features = features.merge(iPhone_6, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  
0                              NaN  
1                              NaN  
2                            213.0  
3                              NaN  
4                              NaN

In [49]:
#iPhone 6s
iPhone_6s = pd.DataFrame()
iPhone_6s = viewed_product_event.loc[viewed_product_event['model'] == 'iPhone 6',['person']]
iPhone_6s= pd.DataFrame(iPhone_6s['person'].value_counts())
iPhone_6s = iPhone_6s.reset_index()
iPhone_6s.rename({'index':'person', 'person':'cantidad de vistas del iphone 6s'}, axis = 1, inplace = True)
iPhone_6s.head()

person  cantidad de vistas del iphone 6s
0  6abd2bf1                               603
1  9bf968c5                               558
2  c86f17ad                               417
3  09ebd175                               341
4  edbdffb9                               318

In [50]:
features = features.merge(iPhone_6s, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  
0                              NaN                               NaN  
1                              NaN                               NaN  
2                            213.0                             213.0  
3                              NaN                               NaN  
4                              NaN                               NaN

**Visitas al sitio segun los dias de la semana**

In [51]:
#Primero veo que dia de la semana hubo mas conversiones
days_convertions = events_data.loc[events_data['event'] == 'conversion',['weekday']]

In [52]:
days_convertions = pd.DataFrame(days_convertions['weekday'].value_counts()/days_convertions.shape[0])
days_convertions.reset_index()

index   weekday
0  Wednesday  0.204626
1   Thursday  0.200959
2    Tuesday  0.196728
3     Friday  0.126075
4     Monday  0.121986
5   Saturday  0.083345
6     Sunday  0.066281

Aproximadamente el 80% de las conversiones se hicieron entre lunes y viernes. 
Veo cuantas veceslos usuarios visitaron la pagina en dias de semana.

In [53]:
weekdays = pd.DataFrame()

In [54]:
weekdays = events_data.loc[events_data['weekday'].isin(['Wednesday','Thursday','Tuesday','Friday','Monday']) & 
                           (events_data['event'] == 'visited site'),['person']]

In [55]:
weekdays.head()

person
2136629  4640420b
2136630  4640420b
2136631  4640420b
2136632  4640420b
2136633  4640420b

In [56]:
weekdays_person_count = pd.DataFrame(weekdays['person'].value_counts())
weekdays_person_count = weekdays_person_count.reset_index()
weekdays_person_count.rename({'index':'person', 'person':'cantidad de vistas entre semana'}, axis = 1, inplace = True)
weekdays_person_count.head()

person  cantidad de vistas entre semana
0  5059f7fd                              281
1  02f14240                              233
2  67bdc946                              157
3  7ac0c607                              154
4  d9777589                              149

In [57]:
features = features.merge(weekdays_person_count, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  
0                              1.0  
1                              5.0  
2                             81.0  
3                              1.0  
4                             20.0

**Cantidad de veces que cada usuario vio un producto el ultimo mes y semana**

In [58]:
views_month = events_data.loc[(events_data['event'] == 'viewed product') & (events_data['date'] >= datetime.date(2018, 5, 1)), ['person']]
views_month= pd.DataFrame(views_month['person'].value_counts())
views_month = views_month.reset_index()
views_month.rename({'index':'person', 'person':'cantidad de veces que vio un producto el ultimo mes'}, axis = 1, inplace = True)
views_month.head()

person  cantidad de veces que vio un producto el ultimo mes
0  6abd2bf1                                               1163  
1  52a4f0b6                                               1062  
2  18489dd5                                               1050  
3  595b9b50                                               1043  
4  f9b343c9                                                971

In [59]:
features = features.merge(views_month, on = 'person', how = 'left')

In [60]:
views_week = events_data.loc[(events_data['event'] == 'viewed product') & (events_data['date'] >= datetime.date(2018, 5, 24)), ['person']]
views_week= pd.DataFrame(views_week['person'].value_counts())
views_week = views_week.reset_index()
views_week.rename({'index':'person', 'person':'cantidad de veces que vio un producto la ultima semana'}, axis = 1, inplace = True)
views_week.head()

person  cantidad de veces que vio un producto la ultima semana
0  595b9b50                                                790     
1  f2408126                                                763     
2  a0b57323                                                614     
3  b5d913c4                                                594     
4  8fb4929e                                                586

In [61]:
features = features.merge(views_week, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  
0                                                NaN       
1                                                NaN       
2                                               43.0       
3                                                7.0       
4                                              235.0

**Cantidad de veces que cada usuario hizo checkouts la ultima semana**

In [62]:
checks_week = events_data.loc[(events_data['event'] == 'checkout') & (events_data['date'] >= datetime.date(2018, 5, 24)), ['person']]
checks_week= pd.DataFrame(checks_week['person'].value_counts())
checks_week = checks_week.reset_index()
checks_week.rename({'index':'person', 'person':'cantidad de checkouts en la ultima semana'}, axis = 1, inplace = True)
checks_week.head()

person  cantidad de checkouts en la ultima semana
0  2a8f49f0                                         41
1  d9777589                                         31
2  c79472e2                                         28
3  19352019                                         24
4  7043f9de                                         24

In [63]:
features = features.merge(checks_week, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  
0                                        NaN  
1                                        NaN  
2                                        NaN  
3                                        1.0  
4                                        NaN

In [64]:
features['cantidad de conversiones por visita'] = (features['cantidad de visitas a la pagina'] / features['cantidad de conversiones'])
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  
0                                  NaN  
1                                  NaN  
2                                  NaN  
3                                  NaN  
4                                  NaN

In [65]:

events_data['new_vs_returning'].value_counts()

Returning    165827
New           38242
Name: new_vs_returning, dtype: int64

**Cantidad de veces que un usuario regreso a la pagina en le ultimo mes**

In [66]:
usuarios_frecuentes = events_data.loc[(events_data['new_vs_returning'] == 'Returning') & 
                                      (events_data['date'] >= datetime.date(2018, 5, 1)),['person']]

In [67]:
users_frec = pd.DataFrame(usuarios_frecuentes['person'].value_counts())

In [68]:
users_frec = users_frec.reset_index()
users_frec.rename({'index':'person', 'person':'cantidad de returnings en el ultimo mes'}, axis = 1, inplace = True)
users_frec.head()

person  cantidad de returnings en el ultimo mes
0  d9777589                                      171
1  e3e70679                                      110
2  0bb45ffd                                      109
3  276a9812                                      101
4  0d43b999                                      100

In [69]:
features = features.merge(users_frec, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  \
0                                  NaN   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   

   cantidad de returnings en el ultimo mes  
0                                      NaN  
1                                      4.0  
2                                     29.0  
3                                      1.0  
4                                     21.0

**Cantidad de veces que volvio a la pagina en la ultima semana**

In [70]:
usuarios_frecuentes_week = events_data.loc[(events_data['new_vs_returning'] == 'Returning') & 
                                      (events_data['date'] >= datetime.date(2018, 5, 24)),['person']]

In [71]:
users_frec_week = pd.DataFrame(usuarios_frecuentes_week['person'].value_counts())
users_frec_week = users_frec_week.reset_index()
users_frec_week.rename({'index':'person', 'person':'cantidad de returnings en la ultima semana'}, axis = 1, inplace = True)
users_frec_week.head()

person  cantidad de returnings en la ultima semana
0  d9777589                                         109
1  e98ce082                                          60
2  069c5926                                          56
3  d0c5aa85                                          52
4  5b63ca60                                          46

In [72]:
features = features.merge(users_frec_week, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  \
0                                  NaN   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   

   cantidad de returnings en el ultimo mes  \
0                                      NaN   
1                                      4.0   
2                                     29.0   
3                                      1.0   
4                                     21.0   

   cantidad de returnings en la ultima s

In [73]:
last_conversion = events_data.loc[(events_data['event'] == 'conversion')]
last_conversion = pd.DataFrame(datetime.date(2018, 6, 1) - last_conversion.groupby('person')['date'].max().fillna(datetime.date(2018,1,1)))
last_conversion = last_conversion.reset_index()
last_conversion['date'] = last_conversion['date'].transform(lambda x: x.days)
last_conversion.rename({'date':'cantidad de dias desde la ultima conversion'}, axis = 1, inplace = True)
last_conversion.head()

person  cantidad de dias desde la ultima conversion
0  000ba417                                            6
1  001001be                                           16
2  001804a2                                            7
3  0019e639                                           10
4  001b0bf9                                           10

In [74]:
features = features.merge(last_conversion, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  \
0                                  NaN   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   

   cantidad de returnings en el ultimo mes  \
0                                      NaN   
1                                      4.0   
2                                     29.0   
3                                      1.0   
4                                     21.0   

   cantidad de returnings en la ultima s

In [75]:
last_view = events_data.loc[(events_data['event'] == 'viewed product')]
last_view = pd.DataFrame(datetime.date(2018, 6, 1) - last_view.groupby('person')['date'].max().fillna(datetime.date(2018,1,1)))
last_view = last_view.reset_index()
last_view['date'] = last_view['date'].transform(lambda x: x.days)
last_view.rename({'date':'cantidad de dias desde la ultima vista a producto'}, axis = 1, inplace = True)
last_view.head()

person  cantidad de dias desde la ultima vista a producto
0  00091926                                                  1
1  00091a7a                                                 67
2  000ba417                                                  6
3  000c79fe                                                  3
4  000e4d9e                                                  2

In [76]:
features = features.merge(last_view, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6  cantidad de vistas del iphone 6s  \
0                              NaN                               NaN   
1                              NaN                               NaN   
2                            213.0                             213.0   
3                              NaN                               NaN   
4                              NaN                               NaN   

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  \
0                                  NaN   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   

   cantidad de returnings en el ultimo mes  \
0                                      NaN   
1                                      4.0   
2                                     29.0   
3                                      1.0   
4                                     21.0   

   cantidad de returnings en la ultima s

In [96]:
last_checkout = events_data.loc[(events_data['event'] == 'checkout')]
last_checkout = pd.DataFrame(datetime.date(2018, 6, 1) - last_checkout.groupby('person')['date'].max().fillna(datetime.date(2018,1,1)))
last_checkout = last_checkout.reset_index()
last_checkout['date'] = last_checkout['date'].transform(lambda x: x.days)
last_checkout.rename({'date':'cantidad de dias desde el ultimo checkout'}, axis = 1, inplace = True)
last_checkout.head()

person  cantidad de dias desde el ultimo checkout
0  0008ed71                                         15
1  00091926                                          6
2  000ba417                                          6
3  000c79fe                                          3
4  000e4d9e                                         16

In [78]:
features = features.merge(last_checkout, on = 'person', how = 'left')
features.head()

person  cantidad de conversiones  cantidad de veces que vio un producto  \
0  4886f805                       NaN                                    4.0   
1  ad93850f                       NaN                                   20.0   
2  0297fc1e                       NaN                                  404.0   
3  2d681dd8                       NaN                                   13.0   
4  cccea85e                       NaN                                  739.0   

   cantidad de visitas a la pagina  cantidad de checkouts  \
0                              1.0                    1.0   
1                              5.0                    1.0   
2                             95.0                    7.0   
3                              2.0                    1.0   
4                             22.0                    1.0   

   visitaron el sitio entre las 13hs y 24hs  cantidad de vistas del J5  \
0                                       1.0                        NaN   
1                                       4.0                        NaN   
2                                      64.0                        NaN   
3                                       2.0                        NaN   
4                                      18.0                        NaN   

   cantidad de vistas del iphone 5s  cantidad de vistas del samsung s6 flat  \
0                               NaN                                     NaN   
1                              17.0                                     NaN   
2                               1.0                                     NaN   
3                               NaN                                     NaN   
4                               2.0                                     1.0   

   cantidad de vistas del iphone 6                    ...                      \
0                              NaN                    ...                       
1                              NaN                    ...                       
2                            213.0                    ...                       
3                              NaN                    ...                       
4                              NaN                    ...                       

   cantidad de vistas entre semana  \
0                              1.0   
1                              5.0   
2                             81.0   
3                              1.0   
4                             20.0   

   cantidad de veces que vio un producto el ultimo mes  \
0                                                4.0     
1                                               20.0     
2                                              133.0     
3                                               13.0     
4                                              739.0     

   cantidad de veces que vio un producto la ultima semana  \
0                                                NaN        
1                                                NaN        
2                                               43.0        
3                                                7.0        
4                                              235.0        

   cantidad de checkouts en la ultima semana  \
0                                        NaN   
1                                        NaN   
2                                        NaN   
3                                        1.0   
4                                        NaN   

   cantidad de conversiones por visita  \
0                                  NaN   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   

   cantidad de returnings en el ultimo mes  \
0                                      NaN   
1                                      4.0   
2                                     29.0   
3                                      1.0   
4                                

In [79]:
features.to_csv('D:/DATOS/RELOADED/csv/exp_features.csv')
#features.to_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/exp_features.csv')

In [92]:
events_data.loc[events_data['event'] == 'searched products'].iloc[4]

timestamp                       2018-05-18 00:28:14
event                             searched products
person                                     d21b8e6b
url                                             NaN
sku                                             NaN
model                                           NaN
condition                                       NaN
storage                                         NaN
color                                           NaN
skus                        12520,12507,12548,12534
search_term                                 moto g5
staticpage                                      NaN
campaign_source                                 NaN
search_engine                                   NaN
channel                                         NaN
new_vs_returning                                NaN
city                                            NaN
region                                          NaN
country                                         NaN
device_type 